In [ ]:
import pandas as pd
import numpy as np
import biom
import re
from biom.util import biom_open  
from biom.table import Table
import os

In [ ]:
meta_main = pd.read_csv('/Users/erfan/Main/oasis/test_augmentation/augmentation/Gevers/feature-frequency-filtered-table.3/meta_data.csv',sep='\t', dtype={'#SampleID': str}, low_memory=False, index_col= '#SampleID')
table_main = biom.load_table('/Users/erfan/Main/oasis/test_augmentation/augmentation/Gevers/feature-frequency-filtered-table.3/feature-table.biom')



In [ ]:
len(meta_main)
len(table_main.ids('observation')), len(table_main.ids('sample'))

In [ ]:
table_main.generated_by

In [ ]:
def read_csv(meta_fp, biom_fp):
    meta = pd.read_csv(meta_fp, sep='\t', dtype={'#SampleID': str}, low_memory=False)
    meta = meta.set_index('#SampleID')
    table = biom.load_table(biom_fp)
    meta = meta.loc[meta.index.isin(table.ids('sample'))] 
    table = table.filter(ids_to_keep=meta.index)
    print(meta_fp, meta.shape, biom_fp, table.shape)
    return meta, table

In [ ]:
def filter_biom(meta, table, min_freq_sum, min_freq):
    
    table = table.filter(axis='sample', ids_to_keep=meta.index,inplace=False)
    
    filter_fn = lambda val, id_, md: val.sum() > min_freq_sum  
    table = table.filter(axis='sample', ids_to_keep=filter_fn, inplace=False)

    filter_fn = lambda val, id_, md: val.sum() > min_freq
    table = table.filter(axis='observation', ids_to_keep=filter_fn, inplace=False)
    
    print("meta_fp", meta.shape, "biom_fp", table.shape)
    meta = meta.loc[meta.index.isin(table.ids('sample'))]
    meta = meta.reindex(table.ids('sample'))
    return meta, table

In [ ]:
def filter_samples_Gevers(meta, table, min_freq_sum, min_freq):
    meta = meta.loc[~(meta.antibiotics == True)]
#     meta = meta.loc[~meta.age.isin(['nan'])]
#     meta.age = meta.age.astype('float')
#     meta = meta.loc[(meta.age >= 2) & (meta.age <= 80)]
    meta = meta.loc[meta.diagnosis.isin(['CD','UC','IC','no'])]
    meta['ibd_cond'] = pd.Series(['HC']*meta.shape[0], index=meta.index)
    meta.loc[meta['diagnosis'].isin(['CD','UC','IC']), 'ibd_cond'] = 'IBD'
    to_remove = ["1939.MGH104824.b", "1939.SKBTI.0952", "1939.SKBTI.0930", "1939.SKBTI.1296", "1939.SKBTI.0220", "1939.SKBTI.0157.b", "1939.SKBTI.0933", "1939.SKBTI.0172.a", "1939.SKBTI.0255.a"]
    meta = meta.loc[~meta.index.isin(to_remove)]
    meta, table = filter_biom(meta, table, min_freq_sum, min_freq)
    
    print("number of IBD samples is", meta.loc[meta['ibd_cond']=='IBD'].shape, table.shape, meta.shape)
    return meta, table

In [ ]:
main_dir = '/Users/erfan/Main/oasis/test_augmentation/augmentation/Gevers/'

In [ ]:
metat1, tablet1 = read_csv(main_dir + '/1939_prep_1177_qiime_20180418-110403.txt', main_dir + '/all.biom')

In [ ]:
meta, table = filter_samples_Gevers(metat1, tablet1,  10000, 2)

In [ ]:
a = set(table.ids('observation')) - set(table_main.ids('observation'))
len(set(table_main.ids('observation')) - set(table.ids('observation')))

In [ ]:
tablet4 = table.filter(axis='observation',ids_to_keep=a,inplace=False)
tablet4 = tablet4.filter(axis='sample', ids_to_keep = set(meta.index),inplace=False)

In [14]:
tablet4.matrix_data

<0x890 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>

In [15]:
meta['IBD'] = 0
meta.loc[meta.ibd_cond == 'IBD','IBD'] = 1

In [16]:
if not os.path.exists(main_dir + 'Final/'):
    os.mkdir(main_dir + 'Final/')

In [17]:
meta.to_csv(main_dir + 'Final/meta_data.csv',sep='\t')

In [18]:
with biom_open(main_dir + 'Final/feature-table.biom', 'w') as f:  
    table.to_hdf5(f, "no filtering on age, no antibiotics, min_freq 0, max_sum 10k")

In [19]:
with open(main_dir + 'Final/species.txt', 'w') as f:
    
    string = "\n".join(table.ids('observation'))
    f.write(string)